In [1622]:
# MODULES

import numpy as np
from random import random, choice, randint
import typing
from functools import partial
from dataclasses import dataclass

In [1623]:
# CONSTANTS

FLOATING_DIGITS = 3  # current: x.xx------------...(quite low precision)
INT_SIZE = 4  # bits current:2**7 = 128 + 1bit(sign)
FLOAT_LENGHT = int(INT_SIZE // 2 + FLOATING_DIGITS * 2)  # XXXX XX XX
BITS2DNA = {"00": "A", "01": "C", "10": "G", "11": "T"}
DNA2BITS = {"A": "00", "C": "01", "G": "10", "T": "11"}
MUTATION = {
    "A": ["C", "G", "T"],
    "C": ["A", "G", "T"],
    "G": ["A", "C", "T"],
    "T": ["A", "C", "G"],
}
MUTATIONS_PER_GEN = 2

In [1624]:
# CLASSES


class Perceptron:
    def __init__(
        self, weights: list[float | int], bias: float | int, activation: typing.Callable
    ):
        self.weights = np.array(weights)
        self.bias = bias
        self.activation = activation

    @classmethod
    def random(cls, weights: int, activation: typing.Callable) -> "Perceptron":
        return cls(
            [(random() * INT_SIZE * 2 - 1) * choice([1, -1]) for _ in range(weights)],
            (random() * INT_SIZE * 2 - 1) * choice([1, -1]),
            activation,
        )

    def __call__(self, inputs: list[int | float]) -> float:
        npinputs = np.array(inputs)
        return self.activation(sum(npinputs * self.weights) + self.bias)


class Network:
    layers: list[list[Perceptron]]
    activation: typing.Callable

    @property
    def shape(self) -> tuple[int, ...]:
        return tuple([len(self.layers[0][0].weights)]) + tuple(
            [len(layer) for layer in self.layers]
        )

    @classmethod
    def random(cls, shape: tuple[int, ...], activattion: typing.Callable) -> "Network":
        n = cls()
        n.activation = activattion
        n.layers = [
            [Perceptron.random(shape[i], activattion) for _ in range(sh)]
            for i, sh in enumerate(shape[1:])
        ]
        return n

    def __call__(self, inputs: list[float | int]):
        for layer in self.layers:
            inputs = [perceptron(inputs) for perceptron in layer]
        return list(map(float, inputs))


class DNA:
    def __init__(self, seq: str = "") -> None:
        self.seq = seq

    def __str__(self) -> str:
        return self.seq

    def __add__(self, other: "DNA") -> "DNA":
        return DNA(self.seq + other.seq)

    def __getitem__(self, key):
        return DNA(self.seq[key])

    def __len__(self):
        return len(self.seq)

    def __iter__(self):
        return iter(self.seq)

    def mutate(self):
        pos = randint(0, len(self.seq) - 1)
        new_seq = list(self.seq)
        new_seq[pos] = choice(MUTATION[new_seq[pos]])
        self.seq = "".join(new_seq)

    def crossover(self, other: "DNA") -> "DNA":
        if len(self) != len(other):
            raise ValueError("DNAs must be same lenght to crossover!")
        point1 = randint(0, len(self) - 3)
        point2 = randint(point1 + 1, len(self) - 1)
        return self[:point1] + other[point1:point2] + self[point2:]

In [1625]:
# FUNCTIONS


def complete_bin(binary: str, digits: int, isneg: bool | None = None) -> str:
    binary = "0" * (digits - len(binary) - (1 if not isneg == None else 0)) + binary
    return (("1" if isneg else "0") if not isneg == None else "") + binary


def int2bin(n: int) -> str:
    return bin(n).split("b")[1]


def bin2dna(binary: str) -> DNA:
    res = DNA()
    for i in range(0, len(binary), 2):
        word = binary[i : i + 2]
        res += DNA(BITS2DNA[word])
    return res


def float2dna(num: int | float) -> DNA:
    num = round(float(num), FLOATING_DIGITS)

    integer, fraction = str(num).split(".")
    integer = int(integer)
    fraction = list(fraction)
    if len(fraction) < FLOATING_DIGITS:
        fraction += ["0"] * (FLOATING_DIGITS - len(fraction))
    fraction = map(int, fraction)

    integer_bin = int2bin(integer)
    fraction = map(int2bin, fraction)

    if len(integer_bin) >= INT_SIZE:
        raise ValueError(f"too big float value {num}")

    integer_complete = complete_bin(integer_bin, INT_SIZE, num < 0)
    onedigitcomplete = partial(complete_bin, digits=4)
    fraction = map(onedigitcomplete, fraction)

    fraction = map(bin2dna, fraction)
    return bin2dna(integer_complete) + sum(fraction, DNA())


def dna2float(dna: DNA) -> float:
    integer_dna = dna[: (INT_SIZE // 2)]
    fraction = dna[(INT_SIZE // 2) :]
    fraction = [fraction[i : i + 2] for i in range(0, 2 * FLOATING_DIGITS, 2)]

    integer_bin = "".join([DNA2BITS[str(some)] for some in integer_dna])
    fraction = map(
        lambda n_dna: "".join([DNA2BITS[letter] for letter in n_dna]), fraction
    )

    integer = int(integer_bin[1:], 2) * (-1 if int(integer_bin[0]) else 1)
    bin2int = partial(int, base=2)

    fraction = map(bin2int, fraction)
    fraction = map(str, fraction)

    return float(f"{integer}.{''.join(list(fraction))}")


def perceptron2dna(perceptron: Perceptron) -> tuple[int, DNA, typing.Callable]:
    weights_dna = DNA()
    for weight in perceptron.weights:
        weights_dna += float2dna(weight)
    return (
        len(perceptron.weights),
        weights_dna + float2dna(perceptron.bias),
        perceptron.activation,
    )


def dna2perceptron(
    number_of_weights: int, dna: DNA, activation: typing.Callable
) -> Perceptron:
    weights_dna = dna[: FLOAT_LENGHT * number_of_weights]
    bias_dna = dna[FLOAT_LENGHT * number_of_weights :]

    weights = []
    for i in range(0, FLOAT_LENGHT * number_of_weights, FLOAT_LENGHT):
        weight_dna = weights_dna[i : i + FLOAT_LENGHT]
        weights.append(dna2float(weight_dna))

    return Perceptron(weights, dna2float(bias_dna), activation)


def network2dna(network: Network) -> tuple[tuple[int, ...], DNA, typing.Callable]:
    network_dna = DNA()
    for layer in network.layers:
        for perceptron in layer:
            network_dna += perceptron2dna(perceptron)[1]
    return (network.shape, network_dna, network.activation)


def dna2network(
    shape: tuple[int, ...], dna: DNA, activation: typing.Callable
) -> Network:
    n = Network()
    layers = []
    pos = 0
    for i, sh in enumerate(shape[1:]):
        layer = []
        pr = shape[i]
        for _ in range(sh):
            perceptron_dna = dna[pos : pos + FLOAT_LENGHT * pr + FLOAT_LENGHT]
            layer.append(dna2perceptron(pr, perceptron_dna, activation))
            pos += FLOAT_LENGHT * pr + FLOAT_LENGHT
        layers.append(layer)
    n.layers = layers
    n.activation = activation
    return n

In [1626]:
@dataclass
class Agent:
    network: Network
    fitness: float = 0.0

    @property
    def dna(self) -> DNA:
        return network2dna(self.network)[1]

    def __call__(self, inputs: list[float | int]) -> list[float]:
        return self.network(inputs)


@dataclass
class Population:
    storage: list[Agent]
    pop_limit: int

    def reproduct(self):
        elite = round(0.5 * ((8 * len(self.storage) + 1) ** 0.5 - 1))
        self.storage.sort(key=lambda x: x.fitness, reverse=True)
        self.storage = self.storage[:elite]
        new_storage = self.storage[:]
        for m in self.storage:
            for m2 in self.storage:
                if m != m2:
                    dna = m.dna.crossover(m2.dna)
                    for _ in range(MUTATIONS_PER_GEN):
                        dna.mutate()
                    new_storage.append(
                        Agent(dna2network(m.network.shape, dna, m.network.activation))
                    )
                    if len(new_storage) >= self.pop_limit:
                        self.storage = new_storage
                        return
        self.storage = new_storage

    @property
    def best(self):
        maxfittness = 0
        maxagent = self.storage[0]
        for a in self.storage:
            if a.fitness > maxfittness:
                maxfittness = a.fitness
                maxagent = a
        return maxagent

In [1627]:
p = Population([], 100)

fn = lambda x,y:x-y

for _ in range(10):
    p.storage.append(Agent(Network.random((2, 1), fn), 0))

for _ in range(1000):
    for a in p.storage:
        a.fitness=0
        x,y = randint(0, 1000), randint(0,1000)
        dist = 1-abs(a([x,y])[0]**2 - fn(x,y)**2)**0.5
        a.fitness = dist
    p.reproduct()


TypeError: <lambda>() missing 1 required positional argument: 'y'

In [ ]:
p.best([5, 2])

[25.0]